In [176]:
import numpy as np
import pandas as pd
import csv

from collections import defaultdict as ddict

from amr_maldi_ml.utilities import ab_name_map

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [177]:
PATH_KSA_FOLDER = '/links/groups/borgwardt/Data/ms_diagnostics/validation/Aarau/'
PATH_FILE_spectra_filelist = PATH_KSA_FOLDER + 'files_position.csv'
PATH_to_Bruker_output = PATH_KSA_FOLDER + 'splits/result_files/'
PATH_FILE_lab_file = PATH_KSA_FOLDER + 'raw/Keime_AB_0101-3108_2018_anonymised_with_Species.csv'
PATH_FILE_ab_matching = PATH_KSA_FOLDER + '../AB-matching.csv'

OUTPUT_FILE = '/links/groups/borgwardt/Data/DRIAMS/DRIAMS-C/id/2018/2018_clean.csv'

In [178]:
num_splits = 15

# (1) match list of spectra files with Bruker output files
# (2) match that with lab results, take only cases where (LabID, species) is unique

In [179]:
%%bash
# replace Umlaute in Aarau script, as this will cause problems later on
#sed -i 's/ä/ae/g' /links/groups/borgwardt/Data/ms_diagnostics/validation/Aarau/raw/Keime_AB_0101-3108_2018_anonymised.csv


### (1) match list of spectra files with output from Bruker report files 

In [180]:
# read-in list of files
for k in range(num_splits):
    
    # read-in filelist_split files
    locals()['filelist_split{}'.format(k)] = pd.read_csv(PATH_KSA_FOLDER+'splits/filelist_files/filelist_split{}.csv'.format(k), sep=';', header=None)
    split_file = locals()['filelist_split{}'.format(k)]
    split_file = split_file.iloc[:,9:]
    split_file.columns = ['Jahr', 'Datum_Kuerzel', 'LaborID', 'Position']

    # check that Tagesnr column has unique entries
    assert len(split_file.LaborID) == len(np.unique(split_file.LaborID))

    locals()['filelist_split{}'.format(k)] = split_file
    assert id(split_file) == id(locals()['filelist_split{}'.format(k)])
    

In [181]:
# how many samples were distributed into the splitfiles
n_sampl = 0
for k in range(num_splits):
    n_sampl+= len(locals()['filelist_split{}'.format(k)])
print(n_sampl)

7862


In [182]:
# read-in reports
n_BrukerID = 0
for k in range(num_splits):
    report = pd.read_csv(PATH_to_Bruker_output+'results_split_{}.csv'.format(k), sep=';', header=None)
    n_BrukerID += len(report)
    
    report.columns = ['LaborID', 'Value','A','Organism_best_match', 'Score1', 'Organism_second_best_match', 'Score2','empty']
    report.drop(['empty'], axis=1, inplace=True)
    
    # check that LaborID column has unique entries
    assert len(report.LaborID) == len(np.unique(report.LaborID))
    locals()['report_{}'.format(k)] = report

print('Number of BrukerID entries {}'.format(n_BrukerID))  

Number of BrukerID entries 7845


In [183]:
# join both tables, clean up for bad Bruker results
n_reduce_all = 0

for k in range(num_splits):  
    report = locals()['report_{}'.format(k)]
    filelist = locals()['filelist_split{}'.format(k)]
    
    spectra_to_species = filelist.set_index('LaborID').join(report.set_index('LaborID'))
    
    # clean up
    n_reduce = len(spectra_to_species)
    print('number entries before clean-up: {}'.format(len(spectra_to_species)))
    spectra_to_species = spectra_to_species[spectra_to_species.Organism_best_match != 'not reliable identification']
    spectra_to_species = spectra_to_species[spectra_to_species.Organism_second_best_match != 'not reliable identification']
    spectra_to_species = spectra_to_species[~spectra_to_species.Organism_best_match.isnull()]
    spectra_to_species = spectra_to_species[~spectra_to_species.Organism_best_match.str.startswith('MIX')]
    spectra_to_species = spectra_to_species[spectra_to_species.Organism_best_match != 'no peaks found']
    
    print('number entries after clean-up: {}\n'.format(len(spectra_to_species)))
    n_reduce -= len(spectra_to_species)
    n_reduce_all += n_reduce   
    locals()['spectra_to_species_{}'.format(k)] = spectra_to_species

print('total number of spectra lost to clean up: {}\n'.format(n_reduce_all))

number entries before clean-up: 3033
number entries after clean-up: 2782

number entries before clean-up: 1976
number entries after clean-up: 1809

number entries before clean-up: 1156
number entries after clean-up: 1008

number entries before clean-up: 671
number entries after clean-up: 581

number entries before clean-up: 387
number entries after clean-up: 321

number entries before clean-up: 242
number entries after clean-up: 194

number entries before clean-up: 150
number entries after clean-up: 115

number entries before clean-up: 93
number entries after clean-up: 78

number entries before clean-up: 58
number entries after clean-up: 42

number entries before clean-up: 37
number entries after clean-up: 25

number entries before clean-up: 20
number entries after clean-up: 16

number entries before clean-up: 15
number entries after clean-up: 12

number entries before clean-up: 10
number entries after clean-up: 8

number entries before clean-up: 7
number entries after clean-up: 5

num

In [184]:
# combine to one spectra_to_species DataFrame
spectra_to_species = spectra_to_species_0
for k in range(1,num_splits):  
    spectra_to_species = pd.concat([spectra_to_species,locals()['spectra_to_species_{}'.format(k)]],axis=0)

print(len(spectra_to_species))
spectra_to_species.head()

7003


,Jahr,Datum_Kuerzel,Position,Value,A,Organism_best_match,Score1,Organism_second_best_match,Score2
LaborID,,,,,,,,,
2806190738,2018,20180620_rm,0_A4,( +++ ),( B ),Enterobacter cloacae,2.41,Enterobacter cloacae,2.26
2806191053,2018,20180620_rm,0_B5,( ++ ),( A ),Staphylococcus aureus,2.27,Staphylococcus aureus,2.23
2806180958,2018,20180620_rm,0_C9,( ++ ),( A ),Candida tropicalis,2,Candida tropicalis,1.98
2806191010,2018,20180620_rm,0_B4,( +++ ),( C ),Klebsiella oxytoca,2.32,Klebsiella oxytoca,2.31
2806180419,2018,20180620_rm,0_C3,( ++ ),( A ),Lautropia mirabilis,2.07,Lautropia mirabilis,2.06


In [185]:
# each df of path to spectra and Bruker DB should have overall unique entries
assert ~any(spectra_to_species.duplicated(keep=False))

# how many ['LaborID','Organism_best_match'] duplicates occur 
x = list(spectra_to_species.reset_index(level=['LaborID']).duplicated(subset=['LaborID','Organism_best_match'], keep=False))
print('Of {} entries in spectra_to_species {} have duplicates.'.format(len(spectra_to_species),sum(x)))

Of 7003 entries in spectra_to_species 3280 have duplicates.


---------
A lot of spectra/samples are duplicated. This seems to occur due to repeated measurements of the same bacterial colony i.e. 4 consecutive well position numbers.

---------

### (2) preprocess lab results, filter and convert to one-line

In [186]:
lab_results = pd.read_csv(PATH_FILE_lab_file, low_memory=False)
print(lab_results.columns)

# drop column with RSI legend
lab_results = lab_results[['Labor-ID', 'Auftraggeb.-ID', 'Auftraggeber', 'Geburtsdatum',
       'Geschlecht', 'Mat-ID', 'Material', 'Status', 'Keim', 'AB-ID',
       'Antibiotikum', 'Sensibilitaet', 'Genus', 'Species']]
lab_results.head()


Index(['Labor-ID', 'Auftraggeb.-ID', 'Auftraggeber', 'Geburtsdatum',
       'Geschlecht', 'Mat-ID', 'Material', 'Status', 'Keim', 'AB-ID',
       'Antibiotikum', 'Sensibilitaet', 'Unnamed: 12', 'Genus', 'Species'],
      dtype='object')


,Labor-ID,Auftraggeb.-ID,Auftraggeber,Geburtsdatum,Geschlecht,Mat-ID,Material,Status,Keim,AB-ID,Antibiotikum,Sensibilitaet,Genus,Species
0,2801010215,200864,PRIVAT-KLINIK IM PARK,60285981e79051e9a1dd5e2ad0a6e968,69691c7bdcc3ce6d5d8a1361f22d04ac,UB,"Urin, Uricult bebrütet",P,Klebsiella pneumoniae variicola,AMC,Amoxicillin-Clavulan,0,Klebsiella,Klebsiella spp.
1,2801010215,200864,PRIVAT-KLINIK IM PARK,60285981e79051e9a1dd5e2ad0a6e968,69691c7bdcc3ce6d5d8a1361f22d04ac,UB,"Urin, Uricult bebrütet",P,Klebsiella pneumoniae variicola,AMP,Ampicillin,2,Klebsiella,Klebsiella spp.
2,2801010215,200864,PRIVAT-KLINIK IM PARK,60285981e79051e9a1dd5e2ad0a6e968,69691c7bdcc3ce6d5d8a1361f22d04ac,UB,"Urin, Uricult bebrütet",P,Klebsiella pneumoniae variicola,CAZ,Ceftazidim,0,Klebsiella,Klebsiella spp.
3,2801010215,200864,PRIVAT-KLINIK IM PARK,60285981e79051e9a1dd5e2ad0a6e968,69691c7bdcc3ce6d5d8a1361f22d04ac,UB,"Urin, Uricult bebrütet",P,Klebsiella pneumoniae variicola,CIP,Ciprofloxacin,0,Klebsiella,Klebsiella spp.
4,2801010215,200864,PRIVAT-KLINIK IM PARK,60285981e79051e9a1dd5e2ad0a6e968,69691c7bdcc3ce6d5d8a1361f22d04ac,UB,"Urin, Uricult bebrütet",P,Klebsiella pneumoniae variicola,CN,Gentamicin,0,Klebsiella,Klebsiella spp.


In [187]:
lab_results_samples = lab_results.iloc[:,[1,2,3,6,7,8,9]]
lab_results_samples = lab_results_samples.drop_duplicates()
print('Number of lab results entries {}'.format(len(lab_results_samples)))


Number of lab results entries 108657


In [188]:
for i in range(len(lab_results)):
    lab_results.iat[i,8] = lab_results.iat[i,8].strip(' ')

In [189]:
# count (LaborID, Keim) that are not unique, while keep multiplied lines for several antibiotics in mind
lab_results_trunc = lab_results.iloc[:,[0,8,9]]
print('Number of lines in lab_results that have a duplicate entry: {}'.format(sum(lab_results_trunc.duplicated(keep=False))))

# remove these duplicated lines
lab_results = lab_results.loc[~lab_results_trunc.duplicated(keep=False)]


Number of lines in lab_results that have a duplicate entry: 308


---
All entries in lab_results are now unique to Lab ID and species. All lines with same Lab ID and species are due to different antibiotics.

---

In [190]:
# make df of all occuring (LaborID, Keim) combinations
LaborID_Keim_combinations = lab_results.drop_duplicates(subset=('Labor-ID','Keim')).drop(columns=lab_results.columns[[1,2,3,4,5,6,7,9,10,11,13]]).reset_index(drop=True)
assert len(LaborID_Keim_combinations.drop_duplicates())==len(LaborID_Keim_combinations)

# make list of all occuring antibiotics
list_antibiotics = list(lab_results['Antibiotikum'].unique())

# ----
# rephrase lab_results to a one-line-per-sample version
# ----

# intialize df
lab_results_oneline = pd.DataFrame(columns=['Labor-ID','Auftraggeb.-ID','Auftraggeber',
                      'Geburtsdatum','Geschlecht','Mat-ID',
                      'Material','Status','Keim','Species']+list_antibiotics)

for row in LaborID_Keim_combinations.itertuples():
    current_labID = row[1]
    current_spec = row[2]
    
    # find all lines with this LabID and Keim
    current_subset = lab_results.loc[lab_results['Labor-ID'] == current_labID].loc[lab_results['Keim'] == current_spec]
    current_antibiotics = list(current_subset['Antibiotikum'])
#     print(current_subset.columns)
    oneline_subset = current_subset[['Labor-ID', 'Auftraggeb.-ID', 'Auftraggeber', 'Geburtsdatum',
       'Geschlecht', 'Mat-ID', 'Material', 'Status', 'Keim', 'Genus', 'Species']].drop_duplicates() 
    assert len(oneline_subset)==1, 'Duplicates present in the subset. {}'.format(oneline_subset)
    
    # add column for each antiobiotic
    for ab in list_antibiotics:
        
        # fill column entry with RSI values if it meets requirements, else empty
        if ab in current_antibiotics:
        
            rsi_value = current_subset.loc[current_subset['Antibiotikum']==ab]['Sensibilitaet'].values[0]
            
            if rsi_value in [0]:
                oneline_subset[ab] = 'S'
            elif rsi_value in [1]:
                oneline_subset[ab] = 'I'
            elif rsi_value in [2]:
                oneline_subset[ab] = 'R'
            else:
                oneline_subset[ab] = ''
        else:
            oneline_subset[ab] = ''

    lab_results_oneline = pd.concat([lab_results_oneline,oneline_subset])
print('number of labfile entries {} (with one entry per sample)'.format(len(lab_results_oneline)))


number of labfile entries 9976 (with one entry per sample)


In [191]:
# if column is completely empty, delete column

len_before = len(lab_results_oneline)

for ab in list_antibiotics: 
    if all(lab_results_oneline[ab].unique() == ''):
        lab_results_oneline.drop(columns=ab, inplace=True)


print('number of excluded samples: {}'.format(len_before - len(lab_results_oneline)))

number of excluded samples: 0


### (3) match that with lab results

In [192]:
# preprocess tables to have the same datatypes
spectra_to_species = spectra_to_species.reset_index(level=['LaborID'])
lab_results_oneline['Labor-ID'] = lab_results_oneline['Labor-ID'].astype('int64')

In [193]:
print(len(spectra_to_species))
print(len(lab_results_oneline))
spectra_to_species_to_AMR = pd.merge(spectra_to_species, lab_results_oneline, 
                                        how='left', 
                                        left_on=['LaborID','Organism_best_match'], 
                                        right_on=['Labor-ID','Species'])
print(spectra_to_species_to_AMR.head())
print(len(spectra_to_species_to_AMR))

7003
9976
      LaborID  Jahr Datum_Kuerzel Position    Value      A  \
0  2806190738  2018   20180620_rm     0_A4  ( +++ )  ( B )   
1  2806191053  2018   20180620_rm     0_B5   ( ++ )  ( A )   
2  2806180958  2018   20180620_rm     0_C9   ( ++ )  ( A )   
3  2806191010  2018   20180620_rm     0_B4  ( +++ )  ( C )   
4  2806180419  2018   20180620_rm     0_C3   ( ++ )  ( A )   

     Organism_best_match Score1 Organism_second_best_match Score2  ...  \
0   Enterobacter cloacae   2.41       Enterobacter cloacae   2.26  ...   
1  Staphylococcus aureus   2.27      Staphylococcus aureus   2.23  ...   
2     Candida tropicalis      2         Candida tropicalis   1.98  ...   
3     Klebsiella oxytoca   2.32         Klebsiella oxytoca   2.31  ...   
4    Lautropia mirabilis   2.07        Lautropia mirabilis   2.06  ...   

   Levofloxacin Minocyclin Ofloxacin Teicoplanin Tigecyclin Tobramycin  \
0                                                                        
1                       

-----
Several lab could not be matched to spectra results file. Reasons identified were different species for the same Lab ID in the lab result file. 

This includes cases of Shigella in the lab file, that were labeled E.Coli by Bruker. 

-----

In [194]:
# analyse entries without match
print(len(spectra_to_species))
print(len(lab_results_oneline) - len(spectra_to_species))

no_matches = spectra_to_species_to_AMR.loc[spectra_to_species_to_AMR['Amoxicillin-Clavulan'].isnull()].iloc[:,:7]
print(no_matches)

mismatching_species = pd.merge(no_matches, lab_results_oneline, 
                                how='left', 
                                left_on=['LaborID'], 
                    #             right_on=['Labor-ID']).iloc[:,[0,6,7,15]]
                                right_on=['Labor-ID']).iloc[:,[6,15]]
print(mismatching_species.Keim.value_counts())



7003
2973
         LaborID  Jahr Datum_Kuerzel Position    Value      A  \
2     2806180958  2018   20180620_rm     0_C9   ( ++ )  ( A )   
4     2806180419  2018   20180620_rm     0_C3   ( ++ )  ( A )   
6     2807304926  2018   20180731_rm    0_H11   ( ++ )  ( A )   
9     2807303407  2018   20180731_rm     0_G6  ( +++ )  ( A )   
10    2804090741  2018    20180411dw     0_C1   ( ++ )  ( A )   
...          ...   ...           ...      ...      ...    ...   
6992  2808040351  2018   20180807_mo    0_G11   ( ++ )  ( C )   
6993  2802150778  2018   20180216_ps     0_B5   ( ++ )  ( B )   
6998  2802150778  2018   20180216_ps     0_B4   ( ++ )  ( B )   
6999  2803211099  2018  20180323_djc     0_B7   ( ++ )  ( A )   
7004  2802160917  2018  20180220_CO2     0_C5   ( ++ )  ( B )   

              Organism_best_match  
2              Candida tropicalis  
4             Lautropia mirabilis  
6     Staphylococcus haemolyticus  
9           Enterococcus faecalis  
10         Streptococcus pyog

In [195]:
spectra_to_species_to_AMR.head()

,LaborID,Jahr,Datum_Kuerzel,Position,Value,A,Organism_best_match,Score1,Organism_second_best_match,Score2,...,Levofloxacin,Minocyclin,Ofloxacin,Teicoplanin,Tigecyclin,Tobramycin,Voriconazol,Ceftazidim-Avibactam,Meropenem-Vaborbacta,Genus
0,2806190738,2018,20180620_rm,0_A4,( +++ ),( B ),Enterobacter cloacae,2.41,Enterobacter cloacae,2.26,...,,,,,,,,,,Enterobacter
1,2806191053,2018,20180620_rm,0_B5,( ++ ),( A ),Staphylococcus aureus,2.27,Staphylococcus aureus,2.23,...,,,,,,,,,,Staphylococcus
2,2806180958,2018,20180620_rm,0_C9,( ++ ),( A ),Candida tropicalis,2,Candida tropicalis,1.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2806191010,2018,20180620_rm,0_B4,( +++ ),( C ),Klebsiella oxytoca,2.32,Klebsiella oxytoca,2.31,...,,,,,,,,,,Klebsiella
4,2806180419,2018,20180620_rm,0_C3,( ++ ),( A ),Lautropia mirabilis,2.07,Lautropia mirabilis,2.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
# remove Nans - no matching LaborID/species in lab results
print('{} entries in spectra_to_species_to_AMR could not be matched to lab results.'.format(sum(spectra_to_species_to_AMR['Minocyclin'].isnull())))

# mismatched (any antibiotic can be used, it would be ' ' if it wasn't present in lab_results_oneline)
spectra_to_species_to_AMR = spectra_to_species_to_AMR.loc[~spectra_to_species_to_AMR['Amoxicillin-Clavulan'].isnull()]

spectra_to_species_to_AMR = spectra_to_species_to_AMR.sort_values(by=['LaborID','Datum_Kuerzel','Position'])

# check for cases where duplicate LaborID/species entries cannot be explained by replicate measurements, 
# i.e. where the well position numbers are not consecutive
x = list(spectra_to_species_to_AMR.duplicated(subset=['LaborID','Organism_best_match'], keep=False))
# print(spectra_to_species_to_AMR.loc[x])

count_reps = spectra_to_species_to_AMR[['LaborID','Datum_Kuerzel','Organism_best_match','Position']].groupby(['LaborID','Datum_Kuerzel','Organism_best_match']).agg(['count'])


2283 entries in spectra_to_species_to_AMR could not be matched to lab results.


In [197]:
# remove lines where [LaborID, Position] is not unique (very few cases)
print(len(spectra_to_species_to_AMR))
spectra_to_species_to_AMR = spectra_to_species_to_AMR.loc[~spectra_to_species_to_AMR.duplicated(subset=['LaborID','Position'], keep=False)]
print(len(spectra_to_species_to_AMR))


4722
4696


In [198]:
spectra_to_species_to_AMR['Oxa/Flucloxacil.'].head()

3784     
6440    R
1341    S
5848    S
3783    S
Name: Oxa/Flucloxacil., dtype: object

### (4) Export table to Aarau_converted.csv

In [199]:
column_names = list(spectra_to_species_to_AMR.columns)
assert id(column_names)!= id(spectra_to_species_to_AMR.columns)

# make list of antibiotics that accour in the column headers spectra_to_species_to_AMR
for col in list(spectra_to_species_to_AMR.columns):
    if col not in list_antibiotics:
        column_names.remove(col)

In [200]:
# convert to export format
Aarau_converted = pd.DataFrame(columns=['species','code']+column_names)

Aarau_converted['species'] = spectra_to_species_to_AMR['Organism_best_match']
Aarau_converted['code'] = spectra_to_species_to_AMR[['LaborID', 'Position']].astype('str').apply(''.join, axis=1)

for col in column_names:
    Aarau_converted[col] = spectra_to_species_to_AMR[col]


# replace each antibiotic column header with the USB equivalent
print(Aarau_converted.columns)
print(ab_name_map[Aarau_converted.columns[2]])
Aarau_converted = Aarau_converted.rename(columns=ab_name_map)

print(Aarau_converted.head())

# export Aarau phenotype file
Aarau_converted.to_csv(OUTPUT_FILE, index=False)

Index(['species', 'code', 'Amoxicillin-Clavulan', 'Ampicillin', 'Ceftazidim',
       'Ciprofloxacin', 'Gentamicin', 'Ceftriaxon', 'Cefuroxim',
       'Nitrofurantoin', 'Fosfomycin', 'Norfloxacin', 'Polymyxin B',
       'Trimethoprim-Sulfame', 'Clarithromycin', 'Clindamycin', 'Doxycyclin',
       'Fusidinsaeure', 'Oxa/Flucloxacil.', 'Penicillin', 'Rifampicin',
       'Vancomycin', 'Linezolid', 'Mupirocin', 'Piperacillin-Tazobac',
       'Metronidazol', 'Moxifloxacin', 'Amikacin', 'Cefepim', 'Imipenem',
       'Azithromycin', 'Erythromycin', 'Cefalotin/Cefazolin', 'Tetracyclin',
       'Novobiocin', 'Pefloxacin', 'Meropenem', 'Daptomycin',
       '5-Fluorocytosin', 'Amphothericin B', 'Amoxicillin', 'Aztreonam',
       'Caspofungin', 'Ceftolozan-Tazobacta', 'Colistin', 'Cefotaxim',
       'Ertapenem', 'Fluconazol', 'Cefoxitin', 'Levofloxacin', 'Minocyclin',
       'Ofloxacin', 'Teicoplanin', 'Tigecyclin', 'Tobramycin', 'Voriconazol',
       'Ceftazidim-Avibactam', 'Meropenem-Vaborbacta'],

In [201]:
ab_name_map

{'AN-Amikacin': 'Amikacin',
 'Amikacin': 'Amikacin',
 'Amikacin 01 mg/l': 'Amikacin_1mg-l',
 'Amikacin 04 mg/l': 'Amikacin_4mg-l',
 'Amikacin 20 mg/l': 'Amikacin_20mg-l',
 'Aminoglykoside': 'Aminoglycosides',
 'Amoxicillin...Clavulansaeure.bei.unkompliziertem.HWI': 'Amoxicillin-Clavulanic acid_uncomplicated_HWI',
 'Amoxicillin-Clavulansaeure.unkompl.HWI': 'Amoxicillin-Clavulanic acid_uncomplicated_HWI',
 'Amoxicillin-Clavulan': 'Amoxicillin-Clavulanic acid',
 'AMC-Amoxicillin/ClavulansÃ¤ure': 'Amoxicillin-Clavulanic acid',
 'Amoxicillin-Clavulansaeure': 'Amoxicillin-Clavulanic acid',
 'Amoxicillin/Clavulansäure': 'Amoxicillin-Clavulanic acid',
 'Amoxicillin...Clavulansaeure': 'Amoxicillin-Clavulanic acid',
 'Amoxicillin/Clavulansaeure': 'Amoxicillin-Clavulanic acid',
 'Amoxicillin': 'Amoxicillin',
 'AMX-Amoxicillin': 'Amoxicillin',
 'Ampicillin': 'Ampicillin',
 'AM-Ampicillin': 'Ampicillin',
 'P-Benzylpenicillin': 'Benzylpenicillin',
 'Benzylpenicillin': 'Benzylpenicillin',
 'Benzylpen

In [202]:
Aarau_converted['Oxacillin'].value_counts()

     3135
S     851
R     710
Name: Oxacillin, dtype: int64